# Домашнее задание 

**1) Приведите еще примеры метрик для оценки рекомендаций/ранжирования (можно взять из интернета, или ваши знания)**

**2) Доделать все функции, где стоит комментарий "сделать дома"**

**Cumulative gain at K (CG@K)** — базовая метрика ранжирования, которая использует простую идею: чем релевантные элементы в этом топе, тем лучше

**WTA** - равна единице, если топовая рекомендация, имеющая самый большой предсказанный рейтинг из просмотренных пользователем, оказалась релевантной, иначе берется ноль.

Метрики на основе ранговой корреляции **Кендэлла, Спирмена**

**PFound** - сумма произведений вероятностей того, что пользователь увидит рекоммендацию на то, что она окажется релевантной. Может применяться для любого списка, который просматривает пользователь

In [19]:
import numpy as np

recommended_list = [143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43] #id товаров
bought_list = [521, 32, 143, 991]

prices_recommended = [100, 90, 10, 450, 50, 37, 99, 120, 34, 100]
prices_bought = [110, 190, 100, 450]

### 1. Hit rate

In [20]:
#был ли хотя бы 1 релевантный товар среди топ-k рекомендованных
def hit_rate_at_k(recommended_list, bought_list, k=5):
    #    сделать дома
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    flags = np.isin(bought_list, recommended_list)
    return (flags.sum() > 0) * 1

In [21]:
hit_rate_at_k(recommended_list, bought_list, 3)

1

### 3. Recall

In [22]:
def recall(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(bought_list)
    

def recall_at_k(recommended_list, bought_list, k=5):
    #    сделать дома
    return recall(recommended_list[:k], bought_list)



def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5):
    #    сделать дома
    recommend_list = np.array(recommended_list)[:k]
    prices_recommended = np.array(prices_recommended)[:k]
    prices_bought = np.array(prices_bought)
    
    flags = np.isin(recommend_list, bought_list)
    
    r1 = np.dot(flags, prices_recommended) 
    r2 = prices_bought.sum()
    recall = r1 / r2
    
    return recall


In [23]:
recall(recommended_list, bought_list)

0.5

In [24]:
print(recall_at_k(recommended_list, bought_list,2))
print(recall_at_k(recommended_list, bought_list,50))

0.25
0.5


In [25]:
print(money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought,1))
print(money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought,3))
print(money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought,4))
print(money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought,5))

0.11764705882352941
0.11764705882352941
0.6470588235294118
0.6470588235294118


# Метрики ранжирования

## AP@k


In [26]:
def precision(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(recommended_list)

def precision_at_k(recommended_list, bought_list, k):
    return precision(recommended_list[:k], bought_list)

In [27]:
precision(recommended_list, bought_list)

0.2

In [28]:
def ap_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    relevant_indexes = np.nonzero(np.isin(recommended_list, bought_list))[0]
    if len(relevant_indexes) == 0:
        return 0
    
    amount_relevant = len(relevant_indexes)
            
    sum_ = sum([precision_at_k(recommended_list, bought_list, k=index_relevant+1) for index_relevant in relevant_indexes])
    return sum_/amount_relevant
  
    

In [29]:
ap_k(recommended_list, bought_list, 5)

0.75

### MAP@k

In [30]:
# теперь список из 3 пользователей
recommended_list_3_users = [[143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43], 
                    [1134, 533, 14, 4, 15, 1543, 1, 99, 27, 3345],
                    [991, 3345, 27, 533, 43, 143, 1543, 156, 1134, 11]
                    ]

bought_list_3_users = [[521, 32, 143],  # юзер 1
                       [143, 156, 991, 43, 11], # юзер 2
                       [1,2]] # юзер 3

In [31]:
def map_k(recommended_list, bought_list, k=5):
    
    result = 0
    for user_num in range(len(recommended_list)):
        result += ap_k(recommended_list[user_num], bought_list[user_num], k)
    return result / len(recommended_list)

In [32]:
map_k(recommended_list_3_users, bought_list_3_users, 5)

0.3333333333333333

### Normalized discounted cumulative gain ( NDCG@k)

In [33]:
def ndcg_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list[:k])
    
    flags = np.isin(recommended_list, bought_list)
    ideal_flags = np.ones(flags.size)
    
    dcg = ( flags[0] + np.sum(flags[1:] / np.log2(np.arange(2, flags.size + 1))) ) / recommended_list.size
    ideal_dcg = ( ideal_flags[0] + np.sum(ideal_flags[1:] / np.log2(np.arange(2, ideal_flags.size + 1))) ) / recommended_list.size
    
    return dcg / ideal_dcg

In [34]:
ndcg_at_k(recommended_list, bought_list, 5)

0.4211582832992166

### Mean Reciprocal Rank ( MRR@k )

In [35]:
def reciprocal_rank(recommended_list, bought_list, k=1):
    recommended_list = np.array(recommended_list)
    bought_list = np.array(bought_list)
    
    amount_user = len(bought_list)
    rr = []
    for i in np.arange(amount_user):    
        relevant_indexes = np.nonzero(np.isin(recommended_list[i][:k], bought_list[i]))[0]
        if len(relevant_indexes) != 0:
            rr.append(1/(relevant_indexes[0]+1))
    
    if len(rr) == 0:
        return 0
    
    return sum(rr)/amount_user

In [36]:
reciprocal_rank(recommended_list_3_users, bought_list_3_users, 5)

C:\Users\sanks\AppData\Local\Temp/ipykernel_8420/1419451445.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  bought_list = np.array(bought_list)


0.3333333333333333